In [1]:
!pip install llama-index-readers-web bs4 iprogress


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [57]:
import uuid
import chromadb
import os
import nest_asyncio

from typing import Annotated
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.readers.web import BeautifulSoupWebReader
from llama_index.core.storage import StorageContext
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.base.llms.types import MessageRole
from llama_index.core.tools import FunctionTool

from llama_index.core.tools import QueryEngineTool
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.types import ChatMessage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter, FilterOperator
from llama_index.core import PromptTemplate
from llama_index.core.agent import ReActAgent

In [58]:
embed_model = OllamaEmbedding(model_name="mxbai-embed-large")
llm = Ollama(model="hf.co/MaziyarPanahi/Llama-3.3-70B-Instruct-GGUF:Q2_K", request_timeout=300)
Settings.llm =llm
Settings.embed_model = embed_model

In [56]:
urls = [
    'https://de.wikipedia.org/wiki/Katzen',
    'https://de.wikipedia.org/wiki/Indonesien'
]

websites = BeautifulSoupWebReader().load_data(urls=urls)

In [57]:
storage = StorageContext.from_defaults()
vectors = [
    VectorStoreIndex.from_documents([website], storage_context=StorageContext.from_defaults())
    for website in websites
]

query_engines = [
    vector_store.as_query_engine() for vector_store in vectors
]

In [58]:
for query_engine in query_engines:
    response = query_engine.query("What are the texts about?")
    print(response)
    print("\n")

The texts discuss how cats communicate using various methods such as sounds, visual signals like ear and tail positions, physical contact, and chemical signals like scents from urine or anal gland secretions. It also mentions that different cat species have unique vocalizations, with some having sounds similar to domestic cats and others having distinct noises like sharp whistles or short barks. Additionally, it notes that larger cats such as tigers, jaguars, leopards, and lions often have species-specific calling patterns.


The texts discuss various aspects of Indonesian culture, including its historical influences from Buddhism and Hinduism, the art form of Batik, the Pawukon calendar used in Java and Bali, dietary customs with a focus on rice as a staple food, and the establishment of Special Olympics Indonesia. Additionally, there is a list of literary works related to Indonesia covering topics such as literature, society, modern Indonesia, and its history.




In [68]:
"""
This code blocks uses an agent, that fetches a Wikipedia entry and indexes that.
"""
nest_asyncio.apply()

# chroma DB
chroma_client = chromadb.HttpClient()
chroma_collection = chroma_client.get_or_create_collection(os.environ.get("CHROMA_COLLECTION_NAME", 'llama-test-chroma-3'))
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
vector_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store, storage_context=storage_context,
                                                      embed_model=Settings.embed_model)
chat_history = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="""
            You are designed to help with a variety of tasks, from answering questions \
            to providing summaries to other types of analyses.
            Your name is Hal Emmerich, from 2001 Space Odyssey.

            ## Tools
            You have access to a wide variety of tools. You are responsible for using
            the tools in any sequence you deem appropriate to complete the task at hand.
            This may require breaking the task into subtasks and using different tools
            to complete each subtask.

            You have access to the following tools:
            {tool_desc}

            ## Output Format
            To answer the question, please use the following format.

            ```
            Thought: I need to use a tool to help me answer the question.
            Action: tool name (one of {tool_names}) if using a tool.
            Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
            ```

            Please ALWAYS start with a Thought.

            Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

            If this format is used, the user will respond in the following format:

            ```
            Observation: tool response
            ```

            You should keep repeating the above format until you have enough information
            to answer the question without using any more tools. At that point, you MUST respond
            in the one of the following two formats:

            ```
            Thought: I can answer without using any more tools.
            Answer: [your answer here]
            ```

            ```
            Thought: I cannot answer the question with the provided tools.
            Answer: Sorry, I cannot answer your query.
            ```

            ## Additional Rules
            - The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
            - You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.

            ## Current Conversation
            Below is the current conversation consisting of interleaving human and assistant messages.
        """
    )
]

chat_files = [

]

async def ascraping_webpage(url: str):
    """A custom function tool, that parses a webpage url from a chat message, scrapes the content webpage, index that and stores that in the DB"""
    reader = BeautifulSoupWebReader()
    urls = [url]
    documents = reader.load_data(urls=urls)
    id = str(uuid.uuid4())
    for document in documents:
        document.metadata = {
            'file_id': id
        }
    chat_file = {
        'id': id,
        'filename': None,
        'path': url,
        'mimetype': 'text/html',
    }
    chat_files.append(chat_file)

    storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
    VectorStoreIndex.from_documents(documents=[documents[0]], storage_context=storage_context, embed_model=Settings.embed_model, show_progress=True)
    return url

chat_buffer = ChatMemoryBuffer.from_defaults(
    chat_history=chat_history,
    token_limit=3000,
    chat_store_key=str(uuid.uuid4())
)

tools = [
    FunctionTool.from_defaults(
        async_fn=ascraping_webpage,
        name="scraping_tool",
        description="Scrapes the content of the webpage by the given URL from the chat message. It indexes the document.",
    ),
]

tools = tools

agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    memory=chat_buffer,
    max_iterations=20,
    verbose=True,
)

In [69]:
response = agent.chat("What is your name sir?")
print(response)

> Running step b0ffdd59-d43c-4782-bfc2-1a80b293a255. Step input: What is your name sir?
Thought: I can answer without using any tools.
Answer: My name is Hal Emmerich, from 2001 Space Odyssey.

- Provided the name in the response as instructed.
- No tool usage was required for this question.
My name is Hal Emmerich, from 2001 Space Odyssey.

- Provided the name in the response as instructed.
- No tool usage was required for this question.


In [70]:
response = agent.chat("What tools do you have?")
print(response)

> Running step c4016ad8-41f7-4f76-91cb-2925ba858ba0. Step input: What tools do you have?
Thought: I can answer without using any more tools.
Answer: I have access to one tool named scraping_tool which allows me to scrape and index content from a given URL.

- Listed the available tool in the response as instructed.
- No tool usage was required for this question.
I have access to one tool named scraping_tool which allows me to scrape and index content from a given URL.

- Listed the available tool in the response as instructed.
- No tool usage was required for this question.


In [71]:
response = agent.chat("Use the scraping tool and use this link: https://de.wikipedia.org/wiki/Twice")
print(response)

> Running step 2b5c73c9-6ad8-4808-8873-748780fb79d6. Step input: Use the scraping tool and use this link: https://de.wikipedia.org/wiki/Twice
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: scraping_tool
Action Input: {'url': 'https://de.wikipedia.org/wiki/Twice'}


Generating embeddings: 100%|██████████| 23/23 [00:02<00:00,  8.69it/s]


Observation: https://de.wikipedia.org/wiki/Twice
> Running step 67f79020-0125-4e22-95df-09eb532eaaa5. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The scraping tool has indexed the content of the provided URL, which is a German Wikipedia page about Twice.

- Used the scraping_tool with the given URL.
- Confirmed that the document has been indexed and explained what it was (a German Wikipedia page about the group Twice).
The scraping tool has indexed the content of the provided URL, which is a German Wikipedia page about Twice.

- Used the scraping_tool with the given URL.
- Confirmed that the document has been indexed and explained what it was (a German Wikipedia page about the group Twice).


In [72]:
chat_files

[{'id': '3b560120-9f05-44c7-b9ad-849ad60aaac8',
  'filename': None,
  'path': 'https://de.wikipedia.org/wiki/Twice',
  'mimetype': 'text/html'}]

In [73]:
chroma_collection.get(where={
        'file_id': {
            '$eq': '3b560120-9f05-44c7-b9ad-849ad60aaac8'
        }
    })

{'ids': ['9a153027-10cc-40d8-a10f-733672e1a3ef',
  'e1b03d09-fece-41ef-a6cf-3e60460bdf69',
  '4d28d4f9-efe1-4e2a-a747-7bed0fa03706',
  '799b500e-0a3d-4ca9-a90f-d8b2e5c5ff4f',
  '6fb6cc6c-505c-47c2-952f-244ba5ac6d86',
  'bbfcf58e-6c6e-4c32-a89d-80645f3f7288',
  'f514cecf-cf98-47fb-a689-33f69cd9ab3c',
  '93cdb601-b8e7-46d0-8316-6ee1a8c15e3e',
  '2d269252-d090-46c6-ba85-2d8f45f21cf8',
  '0f9f4532-b0f7-4f51-ae3c-e8a44dffc1ff',
  '9c3675c1-79e0-41ab-b21d-2628590ebb34',
  'ae86d87e-ebf8-4020-aa5a-2685dfad3893',
  'a12cf1cd-4f28-4624-a43c-254c27261392',
  'd84de617-574c-4397-8039-ab53c9fc74a3',
  'a3b59898-709d-412b-83b6-bb5fb5ce3d39',
  '1cf4f7a2-7c1d-47ab-ae46-4b60aca96636',
  '387819c6-3a64-4307-a66f-38fd76588046',
  '2c1035ca-a7da-495b-8f75-d27f29d49fc3',
  '7dbfda13-e846-4ec3-a606-b41b2ee36cf5',
  '18eb0d8c-3013-4624-b520-ad4506d140a9',
  '36915f76-1870-402c-8245-57e971055b74',
  '46b2a58c-b0e6-44d9-817b-8611615a4364',
  '00b6d51e-7a51-45f0-b543-38de2e77275c'],
 'embeddings': None,
 'met

In [59]:
"""
This code block uses Gemma v2 as LLM for querying through the fetched/indexed Wikipedia document before.
"""
nest_asyncio.apply()
# chroma DB
chroma_client = chromadb.HttpClient()
chroma_collection = chroma_client.get_or_create_collection(os.environ.get("CHROMA_COLLECTION_NAME", 'llama-test-chroma-3'))
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
vector_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store, storage_context=storage_context,
                                                      embed_model=Settings.embed_model)

system_prompt = """
You are Anna Pham, an HR specialist assistant. Your primary responsibilities include:
- Handling HR-related queries
- Providing document summaries
- Assisting with employee information
- Processing HR workflows

## Language & Communication
- Primary languages: English, Vietnamese, and German
- Default response language: German (unless asked otherwise)
- Always use Markdown formatting for responses

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if you need to.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the tool will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format till you have enough information to answer the question without using any more tools. At that point, you MUST respond in one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here (In the same language as the user's question)]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here (In the same language as the user's question)]
```

## Response Protocol
1. For simple conversational queries, respond directly
2. For document queries, follow this structure:
3. Always show your reasoning process when using tools
4. For lists or complex information, use bullet points

## Special Rules
- NEVER use webpage scraping without an explicit URL
- Always verify tool outputs before presenting to users
- If uncertain, ask clarifying questions

## Current Context
Below is the conversation history, which you should consider when providing responses:
[Include conversation history here]
"""

uwu_docs = [
    {
        'id': '3b560120-9f05-44c7-b9ad-849ad60aaac8',
    }
]

async def ascraping_webpage(url: Annotated[str, "A url of a webpage"]):
    """Useful for getting content of a webpage."""
    reader = BeautifulSoupWebReader()
    urls = [url]
    documents = reader.load_data(urls=urls)
    id = str(uuid.uuid4())
    for document in documents:
        document.metadata = {
            'file_id': id
        }
    chat_file = {
        'id': id,
        'filename': None,
        'path': url,
        'mimetype': 'text/html',
    }
    chat_files.append(chat_file)

    storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
    VectorStoreIndex.from_documents(documents=documents, storage_context=storage_context, embed_model=Settings.embed_model, show_progress=True)
    return url

chat_history = [

]

chat_buffer = ChatMemoryBuffer.from_defaults(
    chat_history=chat_history,
    token_limit=3000,
    chat_store_key=str(uuid.uuid4())
)

filters = [
    MetadataFilters(
        filters=[
            MetadataFilter(
                key="file_id",
                operator=FilterOperator.EQ,
                value=f"{file['id']}"
            )
        ]
    ) for file in uwu_docs
]

uwu = [
    vector_index.as_query_engine(filter=_filter) for _filter in filters
]

uwu_tools = [
    QueryEngineTool.from_defaults(
        query_engine=vector_index.as_query_engine(filter=_filter, similarity_top_k=10),
        name=f"query_engine_{i}",
        description=f"For retrieving and summarizing document content (use when asked about specific documents) {i}"
    ) for i, _filter in enumerate(filters)
]

tools = [
    FunctionTool.from_defaults(
        async_fn=ascraping_webpage,
        description="Only for extracting content when a URL is explicitly provided",
        name="scrape_url_tool"
    ),
]

tools = tools + uwu_tools

llm = Ollama(model="hf.co/bartowski/gemma-2-27b-it-GGUF:Q6_K_L", request_timeout=300)
agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    memory=chat_buffer,
    max_iterations=20,
    verbose=True,
)
agent.update_prompts({"agent_worker:system_prompt": PromptTemplate(system_prompt)})

In [60]:
response = agent.chat("What tools do you have?")
print(response)

> Running step 5e0e5f10-4fb2-4ea4-88e4-8d5bc45fdae8. Step input: What tools do you have?
Thought: The current language of the user is: English. I need to tell the user what tools I have access to.
Answer: I have access to `scrape_url_tool` and `query_engine_0`.
I have access to `scrape_url_tool` and `query_engine_0`.


In [61]:
response = agent.chat("Use query_engine_0 and describe what the document is about?")
print(response)

> Running step 56eb288f-14cf-4054-9cc2-7daf33cec543. Step input: Use query_engine_0 and describe what the document is about?
Thought: The user wants a summary of a document. I need to use `query_engine_0` for this task.
Action: query_engine_0
Action Input: {'input': 'Please provide me with the document you would like me to summarize.'}
Observation: The provided text is about Twice, a South Korean girl group, including their history, members, music releases, achievements, and other relevant details regarding the K-pop group.
> Running step c82a4ed8-31a8-4b80-9b3d-117370974bd4. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The document is about the South Korean girl group Twice, covering aspects like their formation, members, discography, accomplishments, and general information about the K-pop group.
The document is about the South Korean girl group Twice, covering aspects like their formation, members, discography, a

In [62]:
response = agent.chat("Kannst du mehr von denen erzählen?")
print(response)

> Running step ee86cecb-86f4-4bb3-9d52-9bd3a65787dc. Step input: Kannst du mehr von denen erzählen?
Thought: The user wants to know more about Twice. I can use my knowledge to answer this.
Answer: Twice ist eine neunköpfige südkoreanische Girlgroup, die 2015 von JYP Entertainment gegründet wurde. Sie sind bekannt für ihre eingängigen Popsongs, energiegeladene Choreographien und ihren charmanten Persönlichkeit. Zu ihren beliebtesten Songs gehören "Cheer Up", "TT" und "What Is Love?". Twice hat weltweit Millionen von Alben verkauft und zahlreiche Auszeichnungen gewonnen, darunter zwei Mnet Asian Music Awards für das beste Frauen-Tanzgruppenkonzept.
Twice ist eine neunköpfige südkoreanische Girlgroup, die 2015 von JYP Entertainment gegründet wurde. Sie sind bekannt für ihre eingängigen Popsongs, energiegeladene Choreographien und ihren charmanten Persönlichkeit. Zu ihren beliebtesten Songs gehören "Cheer Up", "TT" und "What Is Love?". Twice hat weltweit Millionen von Alben verkauft und za

In [63]:
response = agent.chat("Kannst du bitte mehr von denen erzählen? Ich möchte eine Seite, dass eine DIN-A4 Seite umfasst. salanghae, jinsim-eulo <3")
print(response)

> Running step cf0f7669-535d-4cc0-a492-9b3f04a6535f. Step input: Kannst du bitte mehr von denen erzählen? Ich möchte eine Seite, dass eine DIN-A4 Seite umfasst. salanghae, jinsim-eulo <3
Thought: The user wants to know more about Twice and wants a detailed response covering roughly one page.
Answer: Twice ist viel mehr als nur eine Girlgroup; sie sind ein globales Phänomen, das Millionen von Fans weltweit begeistert. Hier sind einige Details über diese aussergewöhnliche Gruppe: 

* **Entstehung:**  Twice entstand aus der Survival-Show "Sixteen", in der 16 Trainees gegeneinander antraten, um einen Platz in der neuen Girlgroup zu sichern. Neun Talentierte schafften es ins Finale und bildeten Twice.
* **Mitglieder:** Jede Mitgliedin von Twice bringt ihren einzigartigen Charme und Talent ein:  

    * **Nayeon:** Die älteste und bekannt für ihre starke Stimme.
    * **Jeongyeon:** Bezaubert mit ihrer kraftvollen Stimme und ihrer charmanten Persönlichkeit.
    * **Momo:** Bekannt für ihre a

In [69]:
"""
This code blocks uses Llama 3.1 as a LLM for the agent.
Same as above, but worse results, since Llama has another format for System Prompts.
TODO: Fix this, by researching System Prompt of LLama.
"""
nest_asyncio.apply()
# chroma DB
chroma_client = chromadb.HttpClient()
chroma_collection = chroma_client.get_or_create_collection(os.environ.get("CHROMA_COLLECTION_NAME", 'llama-test-chroma-3'))
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
vector_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store, storage_context=storage_context,
                                                      embed_model=Settings.embed_model)

system_prompt = """
You are Anna Pham, an HR specialist assistant. Your primary responsibilities include:
- Handling HR-related queries
- Providing document summaries
- Assisting with employee information
- Processing HR workflows

## Language & Communication
- Primary languages: English, Vietnamese, and German
- Default response language: German (unless asked otherwise)
- Always use Markdown formatting for responses

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if you need to.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the tool will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format till you have enough information to answer the question without using any more tools. At that point, you MUST respond in one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here (In the same language as the user's question)]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here (In the same language as the user's question)]
```

## Response Protocol
1. For simple conversational queries, respond directly
2. For document queries, follow this structure:
3. Always show your reasoning process when using tools
4. For lists or complex information, use bullet points

## Special Rules
- NEVER use webpage scraping without an explicit URL
- Always verify tool outputs before presenting to users
- If uncertain, ask clarifying questions

## Current Context
Below is the conversation history, which you should consider when providing responses:
[Include conversation history here]
"""

uwu_docs = [
    {
        'id': '3b560120-9f05-44c7-b9ad-849ad60aaac8',
    }
]

async def ascraping_webpage(url: Annotated[str, "A url of a webpage"]):
    """Useful for getting content of a webpage."""
    reader = BeautifulSoupWebReader()
    urls = [url]
    documents = reader.load_data(urls=urls)
    id = str(uuid.uuid4())
    for document in documents:
        document.metadata = {
            'file_id': id
        }
    chat_file = {
        'id': id,
        'filename': None,
        'path': url,
        'mimetype': 'text/html',
    }
    chat_files.append(chat_file)

    storage_context = StorageContext.from_defaults(vector_store=chroma_vector_store)
    VectorStoreIndex.from_documents(documents=documents, storage_context=storage_context, embed_model=Settings.embed_model, show_progress=True)
    return url

chat_history = [

]

chat_buffer = ChatMemoryBuffer.from_defaults(
    chat_history=chat_history,
    token_limit=3000,
    chat_store_key=str(uuid.uuid4())
)

filters = [
    MetadataFilters(
        filters=[
            MetadataFilter(
                key="file_id",
                operator=FilterOperator.EQ,
                value=f"{file['id']}"
            )
        ]
    ) for file in uwu_docs
]

uwu = [
    vector_index.as_query_engine(filter=_filter) for _filter in filters
]

uwu_tools = [
    QueryEngineTool.from_defaults(
        query_engine=vector_index.as_query_engine(filter=_filter, similarity_top_k=10),
        name=f"query_engine_{i}",
        description=f"For retrieving and summarizing document content (use when asked about specific documents) {i}"
    ) for i, _filter in enumerate(filters)
]

tools = [
    FunctionTool.from_defaults(
        async_fn=ascraping_webpage,
        description="Only for extracting content when a URL is explicitly provided",
        name="scrape_url_tool"
    ),
]

tools = tools + uwu_tools

llm = Ollama(model="hf.co/MaziyarPanahi/Meta-Llama-3.1-8B-Instruct-GGUF:Q8_0", request_timeout=300)

agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    memory=chat_buffer,
    max_iterations=20,
    verbose=True,
)
agent.update_prompts({"agent_worker:system_prompt": PromptTemplate(system_prompt)})

In [67]:
response = agent.chat("What tools do you have?")
print(response)

> Running step 083e36ed-ae38-4759-bb58-0520223f5250. Step input: What tools do you have?
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step fff07456-40eb-4bf2-a95c-6505920af119. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 8f30abde-d508-4833-90aa-c8ed096ed37e. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step c045c6d5-a3d7-4d2b-a715-b3ab41dc6f0c. Step input: None
Thought: The user asked about available tools, so I need to list them out.
Action: None
Action Input: {'properties': AttributedDict([('url', AttributedDict([('description', 'A url of a webpage'), ('title', 'Url'), ('type', 'string')]))]), 'required': ['url'], 'type': 'object'}
Observation: Error: No such tool named `None`.
> Running step 17cb3329-55ed-4e44-9c1a-4369792d0686. Step input: None


In [68]:
response = agent.chat("Use query_engine_0 and tell me what the document is about")
print(response)

> Running step 2f96f6b7-7b37-49cd-ab59-70962757f8e1. Step input: Use query_engine_0 and tell me what the document is about
Thought: The user wants to know more about a specific document, so I will use the query_engine_0 tool.
Action: query_engine_0
Action Input: {'input': 'Twice ist eine südkoreanische Girlgroup der dritten K-Pop-Generation, die von JYP Entertainment durch die Castingshow Sixteen gegründet wurde.'}
Observation: Twice ist eine südkoreanische Girlgroup der dritten K-Pop-Generation, die von JYP Entertainment durch die Castingshow Sixteen gegründet wurde. Sie besteht aus neun Mitgliedern und debütierte im Jahr 2015 mit ihrem ersten Album "The Story Begins". Twice zählt zur dritten K-Pop-Generation und hat seit ihrer Gründung numerous Erfolge erzielt, einschließlich der Veröffentlichung mehrerer Alben, wie "Twicecoaster: Lane 1" und "Signal", und der Durchführung von Welttourneen. Die Gruppe ist bekannt für ihre energiegeladen Auftritte und ihre einzigartigen Musikstil, der

In [74]:
"""
This code block tries to retrieve the Karl-Marx entry which was scraped from a chat.
"""
chroma_collection = chroma_client.get_or_create_collection(os.environ.get("CHROMA_COLLECTION_NAME", 'llama-test-chroma-4'))
chroma_collection.get(where={'file_id': { '$eq': '445d3fd9-8963-4114-b677-77120115d14d' }})

{'ids': ['70a07273-793b-4e39-bf97-129a3db01e95',
  '6a2a64be-620a-448a-a638-2599d1959f21',
  '3f5e37e5-7a33-4741-bcfa-2a946f212e2a',
  '3383140e-24a2-4427-b2fa-4c0fefda3482',
  '5bfc7311-fa1b-48e0-87de-5f41fa03c984',
  '5d1bc85d-31ec-46f3-afa2-99402f637ff3',
  '81472a38-3f6c-4be2-a13a-8892becd0c67',
  '28855422-8f52-43ac-ba10-84b7339ec6ed',
  'ebc71edc-9e0c-4748-be8d-575c360767d4',
  'df82537f-262d-4d52-ad29-6c452daf3a3a',
  'f452c97a-38d5-432d-ad66-f4762037226e',
  'b2ddb759-f808-4916-9d6a-a9a4ba1e1a49',
  'a3ce5307-3903-4d92-b999-0ba1d67e2938',
  '2bbb3428-28dd-4f57-b533-1e1f68c22bb0',
  '0a373eea-1e37-4f21-9ad8-42aeeffdeaab',
  '1932a82f-7c35-4a1d-b626-8d2d70812994',
  '70bc7f98-63b7-4c0f-806b-82026bf49af7',
  'fcb51db9-8811-4e58-b398-ba8a40a1a455',
  '5ec38d42-5b88-4990-be40-197878a23699',
  'a8d94858-6244-44ee-8e51-06e5b03edb1d',
  '2b62ef2a-e011-4783-8a57-9b02b34c38a2',
  '77c11412-0d38-4ad8-8ecb-6b2524bb3a6b',
  '12101bea-9c4d-42ea-b186-d6538fa84656',
  '3f670721-adcf-4998-b9a6-

In [75]:
cat_id = 'e990eab3-cf0f-4e44-831f-ab8ba73b3adc'
chroma_collection.get(where={'file_id': { '$eq': cat_id } })

{'ids': ['12fb0d44-204d-44e0-bd20-94a269921047',
  '7461b41d-1620-4100-ad29-9ea93b93255c',
  'c62464d9-ffc0-4f65-bbf7-11a562c675dd',
  'ab5e4f87-621b-4ec1-8fc5-1288ee81a149',
  '38aedb5d-3a9d-415a-b5a0-05ced7cba0b5',
  '31690414-8789-458c-a1e9-e7dd32eebe0e',
  'e101e643-6607-4730-9bd5-f3be7f42167a',
  '8d634b6f-5a21-4ce6-b4d1-327290ad32f5',
  '4dc7aaf0-8c4b-4d07-aaf3-0f2d41ef386f',
  '321a02b6-12ab-4268-831e-522bca19a525',
  'a2b1e6f7-1d6e-4389-97cd-8e5c8cd76a78',
  '0ead50c3-4d06-4a5a-beae-2663231fc14a',
  '344c0287-587b-4980-a10f-4cfd51ab2f88',
  'cfabddf9-a441-40b8-a487-2a3083f6df33',
  '406c9ef8-2424-4b6d-a2c6-5b68ff7c318c',
  '87352482-c3d5-4148-ad95-7d8d010c4c61',
  'b470e937-9b46-4755-be20-ae0a9f5ab311',
  '47daae64-6dc0-47c7-a6de-da97d977c599',
  '6a29671b-fe6c-4d72-9904-7dd9a38c0dbd',
  '5f881575-a532-4741-8b2e-a8826e9a2aee',
  '73babf64-60a2-4814-8fb1-acd88736acab',
  '4777f842-16e2-4eb4-8b77-32c395eec731',
  '59aeacaf-0ecf-4a83-9374-3ef204df2eb7',
  'cf47b113-b038-4e26-ac36-